In [1]:
import pandas as pd
import holidays
import numpy as np
import requests
from bs4 import BeautifulSoup
from catboost import *
from sklearn.metrics import mean_absolute_error, mean_squared_error
from tqdm.auto import tqdm
import pickle

In [51]:
terms = pd.read_excel('terminal_data_hackathon v4.xlsx')
terms

,TID,longitude,latitude
0,692835,37.646257,55.742062
1,698656,37.666136,55.731231
2,686168,37.675027,55.727031
3,679671,37.669706,55.706824
4,682180,37.596399,55.711151
...,...,...,...
1625,607749,37.138157,56.757843
1626,699046,35.647877,56.226824
1627,637512,35.515147,56.032402
1628,680602,36.543716,55.380772


In [52]:
data = pd.read_excel('terminal_data_hackathon v4.xlsx', 'Incomes')
data.head()

,TID,остаток на 31.08.2022 (входящий),2022-09-01 00:00:00,2022-09-02 00:00:00,2022-09-03 00:00:00,2022-09-04 00:00:00,2022-09-05 00:00:00,2022-09-06 00:00:00,2022-09-07 00:00:00,2022-09-08 00:00:00,...,2022-11-21 00:00:00,2022-11-22 00:00:00,2022-11-23 00:00:00,2022-11-24 00:00:00,2022-11-25 00:00:00,2022-11-26 00:00:00,2022-11-27 00:00:00,2022-11-28 00:00:00,2022-11-29 00:00:00,2022-11-30 00:00:00
0,406136,160000,90000,105000,99000,107000,110000,60000,75000,89000,...,91000,78000,0,165000,0,189000,106000,94000,75000,74000
1,406139,387000,103000,206000,168000,124000,78000,165000,164000,174000,...,164000,153000,151000,157000,206000,182000,123000,138000,112000,179000
2,406145,287000,143000,136000,124000,117000,123000,140000,139000,138000,...,119000,100000,179000,169000,118000,118000,114000,128000,121000,124000
3,406148,355000,50000,73000,53000,65000,75000,100000,53000,52000,...,48000,55000,65000,85000,95000,68000,62000,0,118000,70000
4,406180,597000,96000,82000,71000,72000,86000,55000,55000,75000,...,82000,56000,70000,59000,105000,70000,77000,87000,59000,55000


In [53]:
df_unpivot = pd.melt(data, id_vars='TID', value_vars=data.columns[2:])
data = df_unpivot.sort_values(by=['TID', 'variable'])
data = data.rename(columns={'TID': 'tid', 'variable': 'date', 'value': 'income'})
data['date'] = pd.to_datetime(data['date'])
data.head()

,tid,date,income
0,406136,2022-09-01,90000
1630,406136,2022-09-02,105000
3260,406136,2022-09-03,99000
4890,406136,2022-09-04,107000
6520,406136,2022-09-05,110000


In [54]:
data['train_or_test'] = np.where(data['date'] >= '2022-11-01', 'val', 'train')
data['train_or_test'].value_counts()

train    99430
val      48900
Name: train_or_test, dtype: int64

In [55]:
ru_holidays = holidays.RU()
data['is_holiday'] = data['date'].apply(lambda x: x in ru_holidays)
data['is_holiday'].sum()

1630

In [56]:
data['dayofmonth'] = data.date.dt.day
# df['dayofyear'] = df.date.dt.dayofyear
data['dayofweek'] = data.date.dt.dayofweek
data['month'] = data.date.dt.month
# data['year'] = data.date.dt.year
# df['weekofyear'] = df.date.dt.weekofyear
data['is_month_start'] = (data.date.dt.is_month_start).astype(int)
data['is_month_end'] = (data.date.dt.is_month_end).astype(int)
data.head()

,tid,date,income,train_or_test,is_holiday,dayofmonth,dayofweek,month,is_month_start,is_month_end
0,406136,2022-09-01,90000,train,False,1,3,9,1,0
1630,406136,2022-09-02,105000,train,False,2,4,9,0,0
3260,406136,2022-09-03,99000,train,False,3,5,9,0,0
4890,406136,2022-09-04,107000,train,False,4,6,9,0,0
6520,406136,2022-09-05,110000,train,False,5,0,9,0,0


In [2]:
# Creating sales lag features
def create_sales_lag_feats(df, gpby_cols, target_col, lags):
    gpby = df.groupby(gpby_cols)
    for i in lags:
        df['_'.join([target_col, 'lag', str(i)])] = \
                gpby[target_col].shift(i).values + np.random.normal(scale=1, size=(len(df),)) * 0
    return df

# Creating sales rolling mean features
def create_sales_rmean_feats(df, gpby_cols, target_col, windows, min_periods=2, 
                             shift=1, win_type=None):
    gpby = df.groupby(gpby_cols)
    for w in windows:
        df['_'.join([target_col, 'rmean', str(w)])] = \
            gpby[target_col].shift(shift).rolling(window=w, 
                                                  min_periods=min_periods,
                                                  win_type=win_type).mean().values +\
            np.random.normal(scale=1, size=(len(df),)) * 0
    return df

# Creating sales rolling median features
def create_sales_rmed_feats(df, gpby_cols, target_col, windows, min_periods=2, 
                            shift=1, win_type=None):
    gpby = df.groupby(gpby_cols)
    for w in windows:
        df['_'.join([target_col, 'rmed', str(w)])] = \
            gpby[target_col].shift(shift).rolling(window=w, 
                                                  min_periods=min_periods,
                                                  win_type=win_type).median().values +\
            np.random.normal(scale=1, size=(len(df),)) * 0
    return df

# Creating sales exponentially weighted mean features
def create_sales_ewm_feats(df, gpby_cols, target_col, alpha=[0.9], shift=[1]):
    gpby = df.groupby(gpby_cols)
    for a in alpha:
        for s in shift:
            df['_'.join([target_col, 'lag', str(s), 'ewm', str(a)])] = \
                gpby[target_col].shift(s).ewm(alpha=a).mean().values
    return df

In [58]:
# Converting sales of validation period to nan so as to resemble test period
train = data.copy()
y_val = train.loc[train.train_or_test=='val', 'income'].values.reshape((-1))
y_train = train.loc[train.train_or_test=='train', 'income'].values.reshape((-1))
train.loc[train.train_or_test=='val', 'income'] = np.nan

# # Creating sales lag, rolling mean, rolling median, ohe features of the above train set
train = create_sales_lag_feats(train, gpby_cols=['tid'], target_col='income', 
                               lags=[1, 7, 14, 28])

train = create_sales_rmean_feats(train, gpby_cols=['tid'], 
                                 target_col='income', windows=[1, 3, 7, 14, 28], 
                                 min_periods=1, win_type='triang')

train = create_sales_rmed_feats(train, gpby_cols=['tid'], 
                                 target_col='income', windows=[2, 3, 7, 14, 28], 
                                 min_periods=2, win_type=None)

train = create_sales_ewm_feats(train, gpby_cols=['tid'], 
                               target_col='income', 
                               alpha=[0.9, 0.7, 0.6], 
                               shift=[3, 7, 14, 28])
train['target'] = (train['income']==0)
train = create_sales_lag_feats(train, gpby_cols=['tid'], target_col='target', 
                               lags=[1, 7, 14, 28])

train = create_sales_rmean_feats(train, gpby_cols=['tid'], 
                                 target_col='target', windows=[1, 3, 7, 14, 28], 
                                 min_periods=1, win_type='triang')

train = create_sales_rmed_feats(train, gpby_cols=['tid'], 
                                 target_col='target', windows=[2, 3, 7, 14, 28], 
                                 min_periods=2, win_type=None)

train = create_sales_ewm_feats(train, gpby_cols=['tid'], 
                               target_col='target', 
                               alpha=[0.9, 0.7, 0.6], 
                               shift=[3, 7, 14, 28])


tid_mean = train[train.train_or_test=='train'].groupby('tid')['income'].mean().reset_index().rename(columns={'income': 'tid_mean_income'})
train = train.merge(tid_mean, on='tid', how='left')

# One-Hot Encoding 
# train = one_hot_encoder(train, ohe_cols=['store', 'item', 'month']) 
#,'dayofmonth','weekofyear'

# Final train and val datasets

In [32]:
with open("tid_mean.pkl", 'wb') as f:
    pickle.dump(tid_mean, f)

In [33]:
url = 'http://weatherarchive.ru/Temperature/Moscow/{month}-{year}'
months = ['January', 'February', 'March', 'April', 'May', 'June', 'July',
          'August', 'September', 'October', 'November', 'December']

years = [2022]

In [34]:
url.format(month=months[0], year=2022)

'http://weatherarchive.ru/Temperature/Moscow/January-2022'

In [3]:
def parse_table(table):
    res = {'temp': [],
           'wet': [],
           'p': [],
           'wind': []}
    
    tags = table.findAll('td')
    k = 0
    for tag in tags:
        if tag.find('a') is not None:
            continue
            
        if k == 0:
            k += 1
            res['temp'].append(float(tag.text.replace('°C', '').replace('+','').replace('−','-')))
        elif k == 1:
            k += 1
            res['wet'].append(float(tag.text.replace('%','')))
        elif k == 2:
            k += 1
            res['p'].append(int(tag.text))
        else:
            k = 0
            res['wind'].append(int(tag.text.replace(' м/с', '')))
    return res

def parse_url(url):
    page = requests.get(url)
    soup = BeautifulSoup(page.text, "html.parser")

    tables = soup.findAll('table', class_='smart')
    for table in tables:
        if 'Среднесуточная' in str(table):
            return parse_table(table)

In [37]:
stats = {}
for year in years:
    stats[year] = {}
    for month in tqdm(months):
        stats[year][month] = parse_url(url.format(month=month, year=year))

weather = []
for i, (month, v) in enumerate(stats[2022].items()):
    i = i + 1
    for j, (temp, wet, p, wind) in enumerate(zip(v['temp'], v['wet'], v['p'], v['wind'])):
        j = j + 1
        si = '0' + str(i) if i < 10 else str(i)
        sj = '0' + str(j) if j < 10 else str(j)
        
        weather.append({'date': '2022-{}-{}'.format(si, sj),
                        'temp': temp,
                        'wet': wet,
                        'p': p,
                        'wind': wind})
weather = pd.DataFrame(weather)
weather['date'] = pd.to_datetime(weather['date'])
weather.head()

  0%|          | 0/12 [00:00<?, ?it/s]

,date,temp,wet,p,wind
0,2022-01-01,-3.90,92.25,737,3
1,2022-01-02,-7.80,82.25,744,4
2,2022-01-03,-9.44,87.25,739,3
3,2022-01-04,-9.28,87.75,739,2
4,2022-01-05,-7.40,90.13,734,4


In [38]:
train = train.merge(weather, on='date', how='left')
train.head()

,tid,date,income,train_or_test,is_holiday,dayofmonth,dayofweek,month,is_month_start,is_month_end,...,target_lag_28_ewm_0.7,target_lag_3_ewm_0.6,target_lag_7_ewm_0.6,target_lag_14_ewm_0.6,target_lag_28_ewm_0.6,tid_mean_income,temp,wet,p,wind
0,406136,2022-09-01,90000.0,train,False,1,3,9,1,0,...,NaN,NaN,NaN,NaN,NaN,93770.491803,9.58,68.13,744,4
1,406136,2022-09-02,105000.0,train,False,2,4,9,0,0,...,NaN,NaN,NaN,NaN,NaN,93770.491803,7.59,71.75,748,4
2,406136,2022-09-03,99000.0,train,False,3,5,9,0,0,...,NaN,NaN,NaN,NaN,NaN,93770.491803,7.19,67.50,753,3
3,406136,2022-09-04,107000.0,train,False,4,6,9,0,0,...,NaN,0.0,NaN,NaN,NaN,93770.491803,7.91,65.50,753,2
4,406136,2022-09-05,110000.0,train,False,5,0,9,0,0,...,NaN,0.0,NaN,NaN,NaN,93770.491803,6.41,82.38,752,3


In [39]:
X_val = train.loc[train.train_or_test=='val', :].drop(columns=['train_or_test', 'income', 'target'])
X_train = train.loc[train.train_or_test=='train', :].drop(columns=['train_or_test', 'income', 'target'])
print('Train shape:{}, Val shape:{}'.format(X_train.shape, X_val.shape))

Train shape:(99430, 65), Val shape:(48900, 65)


In [4]:
def smape(preds, target):
    '''
    Function to calculate SMAPE
    '''
    n = len(preds)
    masked_arr = ~((preds==0)&(target==0))
    preds, target = preds[masked_arr], target[masked_arr]
    num = np.abs(preds-target)
    denom = np.abs(preds)+np.abs(target)
    smape_val = (200*np.sum(num/denom))/n
    return smape_val

In [41]:
cat_cols = ['tid']

params_cat = {'n_estimators' : 2000,
          'learning_rate': .02,
          'depth' : 6,
          'use_best_model': True,
          'cat_features' : cat_cols,
          # 'text_features': text_cols,
          # 'train_dir' : '/home/jovyan/work/catboost',
          # 'border_count' : 64,
          # 'l2_leaf_reg' : 1,
          # 'bagging_temperature' : 2,
          # 'rsm' : .1,
          # 'loss_function': 'MultiClass',
          # 'auto_class_weights' : 'Balanced', #try not balanced
          'random_state': 42,
          'eval_metric' : 'MAE'
         }

cat_model = CatBoostRegressor(**params_cat)

In [42]:
cat_model.fit(X_train, y_train, verbose=100, eval_set=(X_val, y_val), early_stopping_rounds=200, plot=True)

MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

0:	learn: 29379.5363450	test: 29660.4378252	best: 29660.4378252 (0)	total: 51.9ms	remaining: 1m 43s
100:	learn: 12995.6951780	test: 14676.0805121	best: 14676.0805121 (100)	total: 5.42s	remaining: 1m 41s
200:	learn: 11649.6760274	test: 14643.7937233	best: 14390.9935418 (135)	total: 10.7s	remaining: 1m 35s
300:	learn: 11407.9726100	test: 14881.5277850	best: 14390.9935418 (135)	total: 15.9s	remaining: 1m 29s
Stopped by overfitting detector  (200 iterations wait)

bestTest = 14390.99354
bestIteration = 135

Shrink model to first 136 iterations.


In [43]:
pred = cat_model.predict(X_val)
mean_absolute_error(y_val, pred), mean_squared_error(y_val, pred), smape(pred, y_val)

(14390.993542795957, 804468281.5142823, 34.467068231860246)

In [46]:
# mean prediction
tid_mean = train[train.train_or_test=='train'].groupby('tid')['income'].mean().reset_index()
mean_pred = X_val.merge(tid_mean, on='tid', how='left')['income']
mean_absolute_error(y_val, mean_pred), mean_squared_error(y_val, mean_pred), smape(mean_pred, y_val)

(13120.229307050186, 686655385.1269174, 31.177421272732385)

In [ ]:
# cross val
dates = X_val['date'].unique()
preds = np.zeros(len(X_val))
for i in tqdm(range(len(dates))):
    train_mask = X_val['date'] < dates[i]
    pred_mask = X_val['date'] == dates[i]
    cat_model = CatBoostRegressor(**params_cat)
    cat_model.fit(pd.concat([X_train, X_val[train_mask]]),
                  np.concatenate([y_train, y_val[train_mask]]),
                  verbose=False,
                  eval_set=(X_val[~train_mask], y_val[~train_mask]), early_stopping_rounds=200)
    preds[pred_mask] = cat_model.predict(X_val[pred_mask])

In [ ]:
mean_absolute_error(y_val, preds), mean_squared_error(y_val, preds), smape(preds, y_val)

### Zero preds

In [102]:
terms = pd.read_excel('terminal_data_hackathon v4.xlsx')
data = pd.read_excel('terminal_data_hackathon v4.xlsx', 'Incomes')
df_unpivot = pd.melt(data, id_vars='TID', value_vars=data.columns[2:])
data = df_unpivot.sort_values(by=['TID', 'variable'])
data = data.rename(columns={'TID': 'tid', 'variable': 'date', 'value': 'income'})
data

,tid,date,income
0,406136,2022-09-01 00:00:00,90000
1630,406136,2022-09-02 00:00:00,105000
3260,406136,2022-09-03 00:00:00,99000
4890,406136,2022-09-04 00:00:00,107000
6520,406136,2022-09-05 00:00:00,110000
...,...,...,...
141809,699664,2022-11-26 00:00:00,0
143439,699664,2022-11-27 00:00:00,44000
145069,699664,2022-11-28 00:00:00,0
146699,699664,2022-11-29 00:00:00,55000


In [32]:
terms = pd.read_excel('terminal_data_hackathon v4.xlsx')
data = pd.read_excel('terminal_data_hackathon v4.xlsx', 'Incomes')

ids, mn, st, mx = [], [], [], []
for i in range(len(data)):
    ids.append(data['TID'].iloc[i])
    lst = []
    for j in range(len(data.columns)):
        if data[data.columns[j]].iloc[i] == 0:
            lst.append(j)
    lst = [lst[i] - lst[i-1] for i in range(1, len(lst))]
    try:
        lst = [np.mean(lst), np.std(lst), np.max(lst)]
    except:
        lst = [0, 0, 0]
    mn.append(lst[0])
    st.append(lst[1])
    mx.append(lst[2])
nw = pd.DataFrame()
nw['tid'] = ids
nw['tid_mean_zero'] = mn
nw['tid_std_zero'] = st
nw['tid_mx_zero'] = mx

df_unpivot = pd.melt(data, id_vars='TID', value_vars=data.columns[2:])
data = df_unpivot.sort_values(by=['TID', 'variable'])
data = data.rename(columns={'TID': 'tid', 'variable': 'date', 'value': 'income'})
data['date'] = pd.to_datetime(data['date'])
data['train_or_test'] = np.where(data['date'] >= '2022-11-01', 'val', 'train')
ru_holidays = holidays.RU()
data['is_holiday'] = data['date'].apply(lambda x: x in ru_holidays)
data['dayofmonth'] = data.date.dt.day
data['dayofweek'] = data.date.dt.dayofweek
data['month'] = data.date.dt.month
data['is_month_start'] = (data.date.dt.is_month_start).astype(int)
data['is_month_end'] = (data.date.dt.is_month_end).astype(int)
data['target'] = (data['income']==0)
url = 'http://weatherarchive.ru/Temperature/Moscow/{month}-{year}'
months = ['January', 'February', 'March', 'April', 'May', 'June', 'July',
          'August', 'September', 'October', 'November', 'December']

years = [2022]

stats = {}
for year in years:
    stats[year] = {}
    for month in tqdm(months):
        stats[year][month] = parse_url(url.format(month=month, year=year))

weather = []
for i, (month, v) in enumerate(stats[2022].items()):
    i = i + 1
    for j, (temp, wet, p, wind) in enumerate(zip(v['temp'], v['wet'], v['p'], v['wind'])):
        j = j + 1
        si = '0' + str(i) if i < 10 else str(i)
        sj = '0' + str(j) if j < 10 else str(j)
        
        weather.append({'date': '2022-{}-{}'.format(si, sj),
                        'temp': temp,
                        'wet': wet,
                        'p': p,
                        'wind': wind})
        
train = data.copy()
y_val = train.loc[train.train_or_test=='val', 'target'].values.reshape((-1))
y_val_incomes = train.loc[train.train_or_test=='val', 'income'].values.reshape((-1))
y_train = train.loc[train.train_or_test=='train', 'target'].values.reshape((-1))
train.loc[train.train_or_test=='val', 'income'] = np.nan

train = create_sales_lag_feats(train, gpby_cols=['tid'], target_col='target', 
                               lags=[1, 7, 14, 28])

train = create_sales_rmean_feats(train, gpby_cols=['tid'], 
                                 target_col='target', windows=[1, 3, 7, 14, 28], 
                                 min_periods=1, win_type='triang')

train = create_sales_rmed_feats(train, gpby_cols=['tid'], 
                                 target_col='target', windows=[2, 3, 7, 14, 28], 
                                 min_periods=2, win_type=None)

train = create_sales_ewm_feats(train, gpby_cols=['tid'], 
                               target_col='target', 
                               alpha=[0.9, 0.7, 0.6], 
                               shift=[3, 7, 14, 28])

train = create_sales_lag_feats(train, gpby_cols=['tid'], target_col='income', 
                               lags=[1, 7, 14, 28])

train = create_sales_rmean_feats(train, gpby_cols=['tid'], 
                                 target_col='income', windows=[1, 3, 7, 14, 28], 
                                 min_periods=1, win_type='triang')

train = create_sales_rmed_feats(train, gpby_cols=['tid'], 
                                 target_col='income', windows=[2, 3, 7, 14, 28], 
                                 min_periods=2, win_type=None)

train = create_sales_ewm_feats(train, gpby_cols=['tid'], 
                               target_col='income', 
                               alpha=[0.9, 0.7, 0.6], 
                               shift=[3, 7, 14, 28])

tid_mean = train[train.train_or_test=='train'][train.income>0].groupby('tid')['income'].mean().reset_index().rename(columns={'income': 'tid_mean_income'})
tid_mean = tid_mean.merge(nw, on='tid', how='left')
with open('zero_aggregation.pkl', 'wb') as f:
    pickle.dump(tid_mean, f)
train = train.merge(tid_mean, on='tid', how='left')

weather = pd.DataFrame(weather)
weather['date'] = pd.to_datetime(weather['date'])
train = train.merge(weather, on='date', how='left')
X_val = train.loc[train.train_or_test=='val', :].drop(columns=['train_or_test', 'income', 'target'])
X_train = train.loc[train.train_or_test=='train', :].drop(columns=['train_or_test', 'income', 'target'])
X_train.head()

C:\Users\gilin\anaconda3\lib\site-packages\numpy\core\fromnumeric.py:3440: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
C:\Users\gilin\anaconda3\lib\site-packages\numpy\core\_methods.py:189: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
C:\Users\gilin\anaconda3\lib\site-packages\numpy\core\_methods.py:262: RuntimeWarning: Degrees of freedom <= 0 for slice
  ret = _var(a, axis=axis, dtype=dtype, out=out, ddof=ddof,
C:\Users\gilin\anaconda3\lib\site-packages\numpy\core\_methods.py:222: RuntimeWarning: invalid value encountered in true_divide
  arrmean = um.true_divide(arrmean, div, out=arrmean, casting='unsafe',
C:\Users\gilin\anaconda3\lib\site-packages\numpy\core\_methods.py:254: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


  0%|          | 0/12 [00:00<?, ?it/s]

C:\Users\gilin\AppData\Local\Temp\ipykernel_27396\2014218281.py:102: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  tid_mean = train[train.train_or_test=='train'][train.income>0].groupby('tid')['income'].mean().reset_index().rename(columns={'income': 'tid_mean_income'})


,tid,date,is_holiday,dayofmonth,dayofweek,month,is_month_start,is_month_end,target_lag_1,target_lag_7,...,income_lag_14_ewm_0.6,income_lag_28_ewm_0.6,tid_mean_income,tid_mean_zero,tid_std_zero,tid_mx_zero,temp,wet,p,wind
0,406136,2022-09-01,False,1,3,9,1,0,NaN,NaN,...,NaN,NaN,102142.857143,11.5,14.762001,44,9.58,68.13,744,4
1,406136,2022-09-02,False,2,4,9,0,0,0.0,NaN,...,NaN,NaN,102142.857143,11.5,14.762001,44,7.59,71.75,748,4
2,406136,2022-09-03,False,3,5,9,0,0,0.0,NaN,...,NaN,NaN,102142.857143,11.5,14.762001,44,7.19,67.50,753,3
3,406136,2022-09-04,False,4,6,9,0,0,0.0,NaN,...,NaN,NaN,102142.857143,11.5,14.762001,44,7.91,65.50,753,2
4,406136,2022-09-05,False,5,0,9,0,0,0.0,NaN,...,NaN,NaN,102142.857143,11.5,14.762001,44,6.41,82.38,752,3


In [33]:
X_train

,tid,date,is_holiday,dayofmonth,dayofweek,month,is_month_start,is_month_end,target_lag_1,target_lag_7,...,income_lag_14_ewm_0.6,income_lag_28_ewm_0.6,tid_mean_income,tid_mean_zero,tid_std_zero,tid_mx_zero,temp,wet,p,wind
0,406136,2022-09-01,False,1,3,9,1,0,NaN,NaN,...,NaN,NaN,102142.857143,11.500000,14.762001,44,9.58,68.13,744,4
1,406136,2022-09-02,False,2,4,9,0,0,0.0,NaN,...,NaN,NaN,102142.857143,11.500000,14.762001,44,7.59,71.75,748,4
2,406136,2022-09-03,False,3,5,9,0,0,0.0,NaN,...,NaN,NaN,102142.857143,11.500000,14.762001,44,7.19,67.50,753,3
3,406136,2022-09-04,False,4,6,9,0,0,0.0,NaN,...,NaN,NaN,102142.857143,11.500000,14.762001,44,7.91,65.50,753,2
4,406136,2022-09-05,False,5,0,9,0,0,0.0,NaN,...,NaN,NaN,102142.857143,11.500000,14.762001,44,6.41,82.38,752,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
148295,699664,2022-10-27,False,27,3,10,0,0,0.0,1.0,...,10815.918139,12924.758990,50615.384615,1.956522,1.102498,8,1.18,85.13,752,2
148296,699664,2022-10-28,False,28,4,10,0,0,1.0,0.0,...,38526.367256,5169.903596,50615.384615,1.956522,1.102498,8,3.80,89.75,753,4
148297,699664,2022-10-29,False,29,5,10,0,0,0.0,1.0,...,15410.546902,41067.961438,50615.384615,1.956522,1.102498,8,6.94,92.13,744,4
148298,699664,2022-10-30,False,30,6,10,0,0,1.0,0.0,...,42164.218761,16427.184575,50615.384615,1.956522,1.102498,8,5.85,75.00,737,8


In [34]:
cat_cols = ['tid']

params_cat = {'n_estimators' : 1000,
          'learning_rate': .03,
          'depth' : 4,
          'use_best_model': True,
          'cat_features' : cat_cols,
          # 'text_features': text_cols,
          # 'train_dir' : '/home/jovyan/work/catboost',
          # 'border_count' : 64,
          # 'l2_leaf_reg' : 1,
          # 'bagging_temperature' : 2,
          # 'rsm' : .1,
          # 'loss_function': 'MultiClass',
          #'auto_class_weights' : 'Balanced', #try not balanced
          'random_state': 42,
          'eval_metric' : 'AUC'
         }

cat_model = CatBoostClassifier(**params_cat)
cat_model.fit(X_train, y_train, verbose=100, eval_set=(X_val, y_val), early_stopping_rounds=200, plot=True)

MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

0:	test: 0.8461521	best: 0.8461521 (0)	total: 53.7ms	remaining: 53.6s
100:	test: 0.9168255	best: 0.9173353 (84)	total: 4.53s	remaining: 40.3s
200:	test: 0.9176645	best: 0.9176645 (200)	total: 9.02s	remaining: 35.9s
300:	test: 0.9185609	best: 0.9187227 (266)	total: 15.5s	remaining: 35.9s
400:	test: 0.9192819	best: 0.9193935 (386)	total: 22.9s	remaining: 34.2s
500:	test: 0.9179211	best: 0.9196323 (446)	total: 30.1s	remaining: 30s
600:	test: 0.9180013	best: 0.9196323 (446)	total: 38.2s	remaining: 25.3s
Stopped by overfitting detector  (200 iterations wait)

bestTest = 0.9196323329
bestIteration = 446

Shrink model to first 447 iterations.


In [35]:
# find best threshold for predictions

tid_mean = train[train.train_or_test=='train'][train.income>0].groupby('tid')['income'].mean().reset_index()

with open('tid_mean.pkl', 'wb') as f:
    pickle.dump(tid_mean, f)
mean_pred = X_val.merge(tid_mean, on='tid', how='left')['income']
bst = 40
bst_thr = 0
for thr in tqdm(range(300, 700)):
    cur = thr/1000
    y_val_preds = cat_model.predict_proba(X_val)[:, 1]>cur
    mean_pred = X_val.merge(tid_mean, on='tid', how='left')['income']
    mean_pred[y_val_preds] = 0
    if smape(mean_pred, y_val_incomes) < bst:
        bst = smape(mean_pred, y_val_incomes)
        bst_thr = cur
mean_pred = X_val.merge(tid_mean, on='tid', how='left')['income']
y_val_preds = cat_model.predict_proba(X_val)[:, 1]>bst_thr
mean_pred[y_val_preds] = 0
print(bst_thr)
mean_absolute_error(y_val_incomes, mean_pred), mean_squared_error(y_val_incomes, mean_pred), smape(mean_pred, y_val_incomes)

C:\Users\gilin\AppData\Local\Temp\ipykernel_27396\3646814006.py:3: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  tid_mean = train[train.train_or_test=='train'][train.income>0].groupby('tid')['income'].mean().reset_index()


  0%|          | 0/400 [00:00<?, ?it/s]

0.357


(12443.525592755343, 649461685.2908142, 27.352996660782765)

In [36]:
from sklearn.metrics import *
print("ACCURACY", accuracy_score(y_val, cat_model.predict(X_val)=='True'))
print("PRECISION", precision_score(y_val, cat_model.predict(X_val)=='True'))
print("RECALL", recall_score(y_val, cat_model.predict(X_val)=='True'))

ACCURACY 0.9544171779141104
PRECISION 0.8062930186823992
RECALL 0.28751753155680226


In [37]:
from catboost import Pool

fi = cat_model.get_feature_importance(Pool(X_val, y_val, cat_features=cat_cols),
                                      prettified=True)
msk = [i for i in range(len(fi)) if fi['Feature Id'].iloc[i] in ['temp', 'wet', 'p', 'wind']]
fi.iloc[msk]

,Feature Id,Importances
21,wet,1.016067
42,temp,0.163948
51,wind,0.075981
55,p,0.038381


In [38]:
with open('catboost_zero.pkl', 'wb') as f:
    pickle.dump(cat_model, f)

In [39]:
X_train

,tid,date,is_holiday,dayofmonth,dayofweek,month,is_month_start,is_month_end,target_lag_1,target_lag_7,...,income_lag_14_ewm_0.6,income_lag_28_ewm_0.6,tid_mean_income,tid_mean_zero,tid_std_zero,tid_mx_zero,temp,wet,p,wind
0,406136,2022-09-01,False,1,3,9,1,0,NaN,NaN,...,NaN,NaN,102142.857143,11.500000,14.762001,44,9.58,68.13,744,4
1,406136,2022-09-02,False,2,4,9,0,0,0.0,NaN,...,NaN,NaN,102142.857143,11.500000,14.762001,44,7.59,71.75,748,4
2,406136,2022-09-03,False,3,5,9,0,0,0.0,NaN,...,NaN,NaN,102142.857143,11.500000,14.762001,44,7.19,67.50,753,3
3,406136,2022-09-04,False,4,6,9,0,0,0.0,NaN,...,NaN,NaN,102142.857143,11.500000,14.762001,44,7.91,65.50,753,2
4,406136,2022-09-05,False,5,0,9,0,0,0.0,NaN,...,NaN,NaN,102142.857143,11.500000,14.762001,44,6.41,82.38,752,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
148295,699664,2022-10-27,False,27,3,10,0,0,0.0,1.0,...,10815.918139,12924.758990,50615.384615,1.956522,1.102498,8,1.18,85.13,752,2
148296,699664,2022-10-28,False,28,4,10,0,0,1.0,0.0,...,38526.367256,5169.903596,50615.384615,1.956522,1.102498,8,3.80,89.75,753,4
148297,699664,2022-10-29,False,29,5,10,0,0,0.0,1.0,...,15410.546902,41067.961438,50615.384615,1.956522,1.102498,8,6.94,92.13,744,4
148298,699664,2022-10-30,False,30,6,10,0,0,1.0,0.0,...,42164.218761,16427.184575,50615.384615,1.956522,1.102498,8,5.85,75.00,737,8


In [40]:
# check inference script
!python predict.py

C:\Users\gilin\anaconda3\lib\site-packages\pandas\core\reshape\merge.py:1219: UserWarning: You are merging on int and float columns where the float values are not equal to their int representation.
  warnings.warn(
C:\Users\gilin\PycharmProjects\hackatons\lct\predict.py:225: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['target'].iloc[j] = msk[j]
C:\Users\gilin\PycharmProjects\hackatons\lct\predict.py:229: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['income'].iloc[j] = tid_mean['income'].iloc[j//(len(data)//1630)]
C:\Users\gilin\PycharmProjects\hackatons\lct\predict.py:225: SettingWithCopyWarning: 
A value 

In [41]:
pd.read_csv('res.csv')

,TID,остаток на 31.08.2022 (входящий),2022-09-01 00:00:00,2022-09-02 00:00:00,2022-09-03 00:00:00,2022-09-04 00:00:00,2022-09-05 00:00:00,2022-09-06 00:00:00,2022-09-07 00:00:00,2022-09-08 00:00:00,...,2022-12-20 00:00:00,2022-12-21 00:00:00,2022-12-22 00:00:00,2022-12-23 00:00:00,2022-12-24 00:00:00,2022-12-25 00:00:00,2022-12-26 00:00:00,2022-12-27 00:00:00,2022-12-28 00:00:00,2022-12-29 00:00:00
0,406136,160000,102142.857143,102142.857143,102142.857143,102142.857143,102142.857143,102142.857143,102142.857143,102142.857143,...,102142.857143,102142.857143,102142.857143,102142.857143,102142.857143,102142.857143,102142.857143,102142.857143,102142.857143,102142.857143
1,406139,387000,160086.206897,160086.206897,160086.206897,160086.206897,160086.206897,160086.206897,160086.206897,160086.206897,...,160086.206897,160086.206897,160086.206897,160086.206897,160086.206897,160086.206897,160086.206897,160086.206897,160086.206897,160086.206897
2,406145,287000,141035.087719,141035.087719,141035.087719,141035.087719,141035.087719,141035.087719,141035.087719,141035.087719,...,141035.087719,141035.087719,141035.087719,141035.087719,141035.087719,141035.087719,141035.087719,141035.087719,141035.087719,141035.087719
3,406148,355000,70344.827586,70344.827586,70344.827586,70344.827586,70344.827586,70344.827586,70344.827586,70344.827586,...,70344.827586,70344.827586,70344.827586,70344.827586,70344.827586,70344.827586,70344.827586,70344.827586,70344.827586,70344.827586
4,406180,597000,78966.101695,78966.101695,78966.101695,78966.101695,78966.101695,78966.101695,78966.101695,78966.101695,...,78966.101695,78966.101695,78966.101695,78966.101695,78966.101695,78966.101695,78966.101695,78966.101695,78966.101695,78966.101695
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1625,699578,47000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1626,699579,236000,65052.631579,65052.631579,0.000000,0.000000,65052.631579,65052.631579,65052.631579,65052.631579,...,65052.631579,65052.631579,65052.631579,65052.631579,0.000000,0.000000,65052.631579,65052.631579,65052.631579,65052.631579
1627,699629,67000,59700.000000,59700.000000,59700.000000,59700.000000,59700.000000,59700.000000,59700.000000,59700.000000,...,59700.000000,59700.000000,59700.000000,59700.000000,59700.000000,59700.000000,59700.000000,59700.000000,59700.000000,59700.000000
1628,699641,278000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
